In [1]:
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout
from keras import Sequential
from keras.preprocessing.image import ImageDataGenerator

# IMAGE PREPROCESSING

In [3]:
train =  ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip= True)
valid = ImageDataGenerator(rescale=1./255)
target = (100,100)
batch = 32


train_set = train.flow_from_directory('dataset/train',
                                      batch_size=batch,
                                      target_size= target,
                                      color_mode= 'grayscale',
                                      class_mode='binary')

valid_set = valid.flow_from_directory('dataset/test',
                                      batch_size=batch,
                                      target_size=target,
                                      color_mode='grayscale',
                                      class_mode='binary')


Found 1234 images belonging to 2 classes.
Found 218 images belonging to 2 classes.


In [5]:
steps_epoch = len(train_set.classes)//batch
validation_steps = len(valid_set.classes) // batch
print(steps_epoch, validation_steps)
print(train_set.class_indices)

38 6
{'Closed': 0, 'Open': 1}


# MODEL CREATION

In [6]:
model = Sequential()
model.add(Conv2D(filters=64, kernel_size=(3,3), input_shape = (100,100,1), activation='relu'))
model.add(MaxPooling2D(pool_size= (2,2)))

model.add(Conv2D(32, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(32, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(16, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(16, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))

model.add(Flatten())
model.add(Dropout(0.5))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(1,activation='sigmoid'))


# MODEL TRAINING

In [7]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [8]:
model.fit_generator(train_set, 
                    validation_data=valid_set, 
                    epochs=10, 
                    steps_per_epoch = steps_epoch, 
                    validation_steps= validation_steps)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_15852\4010835923.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_set, validation_data=valid_set, epochs=10, steps_per_epoch = steps_epoch, validation_steps= validation_steps)


Epoch 1/10
38/38 [==============================] - 45s 1s/step - loss: 0.6812 - accuracy: 0.5433 - val_loss: 0.6132 - val_accuracy: 0.5469
Epoch 2/10
38/38 [==============================] - 25s 653ms/step - loss: 0.5222 - accuracy: 0.7512 - val_loss: 0.4334 - val_accuracy: 0.8177
Epoch 3/10
38/38 [==============================] - 24s 627ms/step - loss: 0.3401 - accuracy: 0.8719 - val_loss: 0.2536 - val_accuracy: 0.8958
Epoch 4/10
38/38 [==============================] - 24s 622ms/step - loss: 0.2263 - accuracy: 0.9176 - val_loss: 0.2655 - val_accuracy: 0.8802
Epoch 5/10
38/38 [==============================] - 24s 629ms/step - loss: 0.2174 - accuracy: 0.9235 - val_loss: 0.1950 - val_accuracy: 0.9323
Epoch 6/10
38/38 [==============================] - 23s 605ms/step - loss: 0.2115 - accuracy: 0.9243 - val_loss: 0.1666 - val_accuracy: 0.9323
Epoch 7/10
38/38 [==============================] - 23s 593ms/step - loss: 0.1848 - accuracy: 0.9326 - val_loss: 0.1528 - val_accuracy: 0.9427
Ep

In [9]:
model.save('models/cnn_2.h5', overwrite=True)

e:\python311\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# PREDICTION

In [31]:
from keras.preprocessing import image
from keras.models import load_model
import numpy as np 

img_path = 'image.jpg'
pred_img = image.load_img(img_path, target_size=(100,100), color_mode='grayscale')

img_arr = image.img_to_array(pred_img)
img_arr = img_arr / 255.0
img_arr = np.expand_dims(img_arr, axis=0)

model = load_model('models/cnn_2.h5')
pred = model.predict(img_arr)
print(pred)

1/1 [==============================] - 0s 216ms/step
[[0.6454252]]


In [32]:
if (pred < 0.5):
    pred = 0
else:
    pred = 1

print(pred)

1
